In [1]:
# Import libraries
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import time

today = datetime.date.today()
print(today)

2024-08-13


In [3]:
# Functions to scrape the Title, Price, Rating, Reviews, and Monthly Sales of a Product on the Product Page
def get_title(soup):
    try:
        product_title = soup.find("span", attrs={"id":'productTitle'})
        title_value = product_title.text
        title_string = title_value.strip()
    
    except AttributeError:
        title_string = ""
    
    return title_string
    

def get_price(soup):
    price = soup.find(class_='a-offscreen')
    if price:
        price_text = price.text.strip()
        try:
            price = float(price_text[2:])
            return price
        except ValueError:
            return "N/A"
    else:
        return "N/A"

def get_rating(soup):
    rating = soup.find(class_='a-icon-alt')
    if rating:
        ratings = rating.text.split(' out of 5 stars')
        ratings = ratings[0]
        rat = ratings.strip()
        return rat
    else:
        return "N/A"
    
def get_reviews(soup):
    reviews_element = soup.find("span", attrs={'id':'acrCustomerReviewText'})
    if reviews_element:
        reviews = reviews_element.text.split(' ratings')
        reviews = reviews[0]
        rev = reviews.strip()
        return rev
    else:
        return "N/A"

def get_sales(soup):
    sales = soup.find(class_='a-size-small social-proofing-faceout-title-text')
    if sales:
        return sales.text
    else:
        return "N/A"

In [4]:
# Scraping the amazon website
# Base URL of the site (without the page number)
base_url = 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723484528&sprefix=sun%2Caps%2C833&ref=sr_pg_1'

# Header used to get request to scrape the webpage
headers = ({"User-Agent": " ", 'Accept-Language': 'en-US en;=0.5'})

# Define the number of pages you want to scrape
num_pages = 7

# Create a dictionary to store product details
d = {'title': [], 'price': [], 'rating': [], 'reviews': [], 'monthly_sales': [], 'URL': []}

# Loop through each page
for page in range(1, num_pages +1):
    url = f'{base_url}{page}'

    response = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    links = soup.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})
    
    for link in links:
        product_url = "https://www.amazon.sg" + link.get('href')
        new_product = requests.get(product_url, headers=headers)
        product_soup = BeautifulSoup(new_product.content, "html.parser")

        d['title'].append(get_title(product_soup))
        d['price'].append(get_price(product_soup))
        d['rating'].append(get_rating(product_soup))
        d['reviews'].append(get_reviews(product_soup))
        d['monthly_sales'].append(get_sales(product_soup))
        d['URL'].append(product_url)
        
        time.sleep(2)

In [5]:
# Create DataFrame
amazon_df = pd.DataFrame.from_dict(d)
amazon_df.tail()

,title,price,rating,reviews,monthly_sales,URL
283,,N/A,N/A,N/A,N/A,https://www.amazon.sg/MDSolarSciences-Mineral-...
284,,N/A,N/A,N/A,N/A,https://www.amazon.sg/Cetaphil-SPF50-Liposomal...
285,"Banana Boat Sport Ultra Sunscreen Lotion, SPF ...",17.77,4.7,"1,621",N/A,https://www.amazon.sg/Banana-Sport-Ultra-Sunsc...
286,Blue Lizard Baby Mineral Sunscreen with Zinc O...,17.82,Previous page,N/A,N/A,https://www.amazon.sg/Mineral-Sunscreen-Resist...
287,Shiseido Water-Resistant UV Protection Clear S...,N/A,4.6,886,N/A,https://www.amazon.sg/Shiseido-Water-Resistant...


In [25]:
import csv

# Creating the csv file
with open('AmazonWebScraper_SunscreenSingapore.csv', 'w', newline='', encoding='UTF8') as f:
    amazon_df.to_csv(f, index=False)

In [26]:
# Appending data to the csv
with open('AmazonWebScraper_SunscreenSingapore.csv', 'a', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(amazon_df.columns)
    writer.writerows(amazon_df.values)